In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

data = fetch_california_housing()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

base_model = RandomForestRegressor(random_state=42)


In [2]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print("Best params:", grid_search.best_params_)
print("Best CV R²:", grid_search.best_score_)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best params: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}
Best CV R²: 0.8004439227116409


In [3]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

print("Test R²:", r2_score(y_test, y_pred))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


Test R²: 0.8048522204167272
Test RMSE: 0.5056910148002403


In [4]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search.fit(X_train_scaled, y_train)

print("Best params (Randomized):", random_search.best_params_)
print("Best CV R² (Randomized):", random_search.best_score_)


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best params (Randomized): {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 264}
Best CV R² (Randomized): 0.8018106144117718


In [5]:
print("GridSearch best R²:", grid_search.best_score_)
print("RandomizedSearch best R²:", random_search.best_score_)


GridSearch best R²: 0.8004439227116409
RandomizedSearch best R²: 0.8018106144117718
